In [ ]:
import sys

sys.path.append("../../..")

from research.config import set_project_root

set_project_root(levels_up=3, print_root=True)

In [ ]:
import pandas as pd
import numpy as np
from finance_byu.summarize import summary
from scipy.stats import zscore
from research.datasets import RussellDaily, CRSPDaily

In [ ]:
# Read in crsp daily data
df = CRSPDaily().df

# Drop columns
df = df.drop(columns=["shrcd", "exchcd", "shrout"])

# Clean date variables
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month

# In sample
start = np.datetime64("2000-01-01")
end = np.datetime64("2020-12-31")

df = df[(df["date"] >= start) & (df["date"] <= end)]

df = df.reset_index(drop=True)

df

In [ ]:
# Read in benchmark data
bmk = RussellDaily().df

# Merge
df = df.merge(bmk, on="date", how="inner", suffixes=("", "_mkt"))

df = df.drop_duplicates().reset_index(drop=True)

df

In [ ]:
# Compute residual (alpha) for each permno on a rolling year window

year_window = 252

df["ret_mkt_cov"] = (
    df.groupby("permno")[["ret", "ret_mkt"]]
    .apply(lambda x: x["ret"].rolling(year_window, year_window).cov(x["ret_mkt"]))
    .reset_index(drop=True)
)

df["mkt_var"] = df.groupby("permno")["ret_mkt"].rolling(year_window).var().reset_index(drop=True)

df["beta"] = df["ret_mkt_cov"] / df["mkt_var"]

df["res"] = df["ret"] - df["ret_mkt"] * df["beta"]

df

In [ ]:
# Calculate reversal signal
month_window = 22

# Log Returns
df["logret"] = np.log1p(df["ret"])

# Momentum from t-6 to t-1
df["rev"] = (
    df.groupby("permno")["logret"].rolling(month_window, month_window).sum().reset_index(drop=True)
)
df["rev"] = df.groupby("permno")["rev"].shift(1)  # lag the reversal feature

df

In [ ]:
# Compute Alphas

df["ic"] = (
    df.groupby("permno")[["rev", "res"]]
    .apply(lambda group: group["rev"].rolling(month_window, month_window).corr(group["res"]))
    .reset_index(level=0, drop=True)
)

df["res_vol"] = (
    df.groupby("permno")["res"].rolling(month_window, month_window).std().reset_index(drop=True)
)

df = df.dropna().reset_index(drop=True)
df["z_rev"] = df.groupby("date")["rev"].apply(zscore).reset_index(drop=True)

df["alpha"] = df["ic"] * df["res_vol"] * df["z_rev"]

df

In [ ]:
alphas = df.pivot(columns="permno", values="alpha", index="date")

alphas